<a href="https://colab.research.google.com/github/deoa17/Analisis-de-sentiminto-de-tweets-usando-Tweepy-y-Nltk/blob/master/Ejercicios_de_Modelos_Clasificacion_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ejercicios de Modelo de clasificación Naive Bayes**

1.   Clasificación de palabras (por género de nombre)
2.   Clasificación de correos (Ham o Spam)



## Ejercicio de práctica: Clasificación de palabras por género

**Objetivo:** Construye un classificador de nombres en español usando el siguiente dataset: 
https://github.com/jvalhondo/spanish-names-surnames

1. **Preparación de los datos**: con un `git clone` puedes traer el dataset indicado a tu directorio en Colab, luego asegurate de darle el formato adecuado a los datos y sus features para que tenga la misma estructura del ejemplo anterior con el dataset `names` de nombres en ingles. 

* **Piensa y analiza**: ¿los features en ingles aplican de la misma manera para los nombres en español?

In [2]:
!git clone https://github.com/jvalhondo/spanish-names-surnames

Cloning into 'spanish-names-surnames'...
remote: Enumerating objects: 36, done.
remote: Total 36 (delta 0), reused 0 (delta 0), pack-reused 36
Unpacking objects: 100% (36/36), done.


In [3]:
# Cargamos los archivos csv con numpy para tenerlos en listas
import numpy as np
import nltk, random
tag_men = np.genfromtxt('/content/spanish-names-surnames/male_names.csv', skip_header=1, delimiter=',', dtype=('U20','i8','f8'))
tag_women = np.genfromtxt('/content/spanish-names-surnames/female_names.csv', skip_header=1, delimiter=',', dtype=('U20','i8','f8'))

In [4]:
# Verificamos
print(tag_men[0])
print(tag_women[0])

('ANTONIO', 715215, 54.6)
('MARIA CARMEN', 668639, 54.5)


In [ ]:
# Juntamos las listas de hombres y mujeres
f_set = [(name[0],'male') for name in tag_men] + [(name[0],'female') for name in tag_women]

# Hacemos un shuffle
random.shuffle(f_set)

2. **Entrenamiento y performance del modelo**: usando el classificador de Naive Bayes de NLTK entrena un modelo sencillo usando el mismo feature de la última letra del nombre, prueba algunas predicciones y calcula el performance del modelo. 

In [ ]:
# Entrenamos teniendo en cuenta nuestro primer atributo (Revisando la ultima letra del nombre)
def atributo_uno(palabra):
	return {'ultima_letra': palabra[-1].lower()}

In [ ]:
from sklearn.model_selection import train_test_split

f_un_atributo = [(atributo_uno(n), g) for (n, g) in f_set]

#Usamos el 80% de los datos para train y 20% para test
f_un_atributo_train, f_un_atributo_test = train_test_split(f_un_atributo, test_size=0.20, random_state=45)

In [ ]:
# Entrenamos
classifier_1 = nltk.NaiveBayesClassifier.train(f_un_atributo_train)

In [ ]:
#Probamos
classifier_1.classify(atributo_uno('daniela'))

'female'

In [ ]:
# Sacamos el accuracy
print(nltk.classify.accuracy(classifier_1, f_un_atributo_test))

0.7919537900283745


3. **Mejores atributos:** Define una función como `atributos2()` donde puedas extraer mejores atributos con los cuales entrenar una mejor version del clasificador. Haz un segundo entrenamiento y verifica como mejora el performance de tu modelo. ¿Se te ocurren mejores maneras de definir atributos para esta tarea particular?

In [ ]:
# Agregamos mas atributos para ver cuanto mejora el accuracy
def atributos2(nombre):
    atrib = {}
    atrib["ultima_letra"] = nombre[-1].lower() #Ultima letra
    atrib["ultimas_dos_letra"] = nombre[-1:-5:-1].lower() #ultimas 4 letras
    return atrib

f_varios_atributo = [(atributos2(n), g) for (n, g) in f_set]

#Usamos el 80% de los datos para train y 20% para test
f_varios_atributo_train, f_varios_atributo_test = train_test_split(f_varios_atributo, test_size=0.20, random_state=45)

In [ ]:
# Entrenamos
classifier_2 = nltk.NaiveBayesClassifier.train(f_varios_atributo_train)

In [ ]:
# Sacamos accuracy
print(classifier_2.classify(atributos('Juan')))
print(nltk.classify.accuracy(classifier_2, f_varios_atributo_test))

male
0.9079854073773814


# Clasificación de documentos (email spam o no spam)

### Ejercicio de práctica


¿Como podrías construir un mejor clasificador de documentos?

0. **Dataset más grande:** El conjunto de datos que usamos fue muy pequeño, considera usar los archivos corpus que estan ubicados en la ruta: `datasets/email/plaintext/` 

1. **Limpieza:** como te diste cuenta no hicimos ningun tipo de limpieza de texto en los correos electrónicos. Considera usar expresiones regulares, filtros por categorias gramaticales, etc ... . 

---

Con base en eso construye un dataset más grande y con un tokenizado más pulido. 

In [ ]:
!git clone https://github.com/pachocamacho1990/datasets
import nltk
import pandas as pd
import numpy as np
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize

fatal: destination path 'datasets' already exists and is not an empty directory.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
# Descomprimir ZIP
import zipfile
fantasy_zip = zipfile.ZipFile('/content/datasets/email/plaintext/corpus1.zip')
fantasy_zip.extractall('/content/datasets/email/plaintext')
fantasy_zip.close()

In [ ]:
# Creamos un listado de los archivos dentro del Corpus1 ham/spam
from os import listdir

path_ham = "/content/datasets/email/plaintext/corpus1/ham/"
filepaths_ham = [path_ham+f for f in listdir(path_ham) if f.endswith('.txt')]

path_spam = "/content/datasets/email/plaintext/corpus1/spam/"
filepaths_spam = [path_spam+f for f in listdir(path_spam) if f.endswith('.txt')]

In [ ]:
# Creamos la funcion para tokenizar y leer los archivos 

def abrir(texto):
  with open(texto, 'r', errors='ignore') as f2:
    data = f2.read()
    data = word_tokenize(data)
  return data

# Creamos la lista tokenizada del ham
list_ham = list(map(abrir, filepaths_ham))
# Creamos la lista tokenizada del spam
list_spam = list(map(abrir, filepaths_spam))

In [ ]:
nltk.download('stopwords')

# Separamos las palabras mas comunes
all_words = nltk.FreqDist([w for tokenlist in list_ham+list_spam for w in tokenlist])
top_words = all_words.most_common(250)

# Agregamos Bigramas
bigram_text = nltk.Text([w for token in list_ham+list_spam for w in token])
bigrams = list(nltk.bigrams(bigram_text))
top_bigrams = (nltk.FreqDist(bigrams)).most_common(250)


def document_features(document):
    document_words = set(document)
    bigram = set(list(nltk.bigrams(nltk.Text([token for token in document]))))
    features = {}
    for word, j in top_words:
        features['contains({})'.format(word)] = (word in document_words)

    for bigrams, i in top_bigrams:
        features['contains_bigram({})'.format(bigrams)] = (bigrams in bigram)
  
    return features

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Juntamos las listas indicando si tienen palabras de las mas comunes
import random
fset_ham = [(document_features(texto), 0) for texto in list_ham]
fset_spam = [(document_features(texto), 1) for texto in list_spam]
fset = fset_spam + fset_ham[:1500]
random.shuffle(fset)

# Separamos en las listas en train y test
from sklearn.model_selection import train_test_split
fset_train, fset_test = train_test_split(fset, test_size=0.20, random_state=45)

2. **Validación del modelo anterior:**  
---

una vez tengas el nuevo conjunto de datos más pulido y de mayor tamaño, considera el mismo entrenamiento con el mismo tipo de atributos del ejemplo anterior, ¿mejora el accuracy del modelo resultante?

In [ ]:
# Entrenamos el programa
classifier = nltk.NaiveBayesClassifier.train(fset_train)

# Probamos y calificamos
classifier.classify(document_features(list_ham[34]))
print(nltk.classify.accuracy(classifier, fset_test))


0.8966666666666666
